In [1]:
import pandas as pd
import numpy as np
import requests
import datetime
from bs4 import BeautifulSoup
import json
from os import listdir
from os.path import isfile, join
from bs4 import Comment
import time
import os

In [4]:
# folder yang akan digunakan

# folder ini berisi file json judul artikel dari detik finance
if not os.path.exists('data/json_detik_finance'):
    os.makedirs('data/json_detik_finance')

# folder berisi isi berita per tanggal dari detik finance
if not os.path.exists('data/isi_berita_detik'):
    os.makedirs('data/isi_berita_detik')

## Scraping Judul

In [2]:
def scrape_page(halaman, tanggal):
    """Melakukan request kehalaman detik finance 
    pada tanggal tertentu dengan halaman tertentu"""
    
    URL = "https://finance.detik.com/indeks/{}?date={}".format(halaman, tanggal)
    page = requests.get(URL)
    return BeautifulSoup(page.content, 'html.parser')

def extract_informasi_berita(berita):
    """Mengambil informasi judul berita, link, dan waktu publish"""
    
    media_text = berita.find("div", class_="media__text")

    media_title = media_text.find("h3", class_="media__title").find("a")
    title = media_title.text.strip()
    link = media_title["href"]
    kategori = link.split("/")[3].replace("-", " ")

    media_date = media_text.find("div", class_="media__date").find("span")["d-time"]
    media_subtitle = media_text.find("h2", class_="media__subtitle")
    if media_subtitle is not None:
        media_subtitle = media_subtitle.text.lower()

    return {"judul" : title, "sub_judul": media_subtitle, "kategori" : kategori, "timestamp_publish" : media_date,"link" : link}

def scrape_tanggal(tanggal):
    """Scraping pada tanggal tertentu
    """
    
    is_first = True
    halaman_sekarang = 1
    jumlah_halaman = 100
    beritas = []

    # Dalam satu tanggal bisa saja beritanya banyak sehingga dipecah menjadi
    # beberapa halaman (pagination)
    while halaman_sekarang <= jumlah_halaman:
        soup = scrape_page(halaman_sekarang, tanggal)

        list_berita = soup.find_all("article", class_="list-content__item")    
        if len(list_berita) > 0:
            for berita in list_berita:
                beritas.append(extract_informasi_berita(berita))

            # Jika halaman pertama, cek ada berapa halaman dengan
            # melihat pada jumlah pagination dibagian bawah halaman
            # kemudian update jumlah halaman sehingga loop 'while' bisa berakhir
            if is_first:
                is_first = False
                list_pagination = soup.find_all("a", class_="pagination__item")
                idx_last_page = len(list_pagination) - 2
                jumlah_halaman = int(list_pagination[idx_last_page].text)
        elif is_first:
            jumlah_halaman = 0

        halaman_sekarang += 1
    return beritas, jumlah_halaman

In [3]:
def generate_date_range(start, end=None):
    """Generate tanggal dengan rentang tertentu. Parameter
    'Start' dan 'End' harus berformat d-m-Y. Misalnya 27-01-2001
    """
    start = datetime.datetime.strptime(start, "%d-%m-%Y")
    if end is None:
        end = datetime.datetime.today()
    else:
        end = datetime.datetime.strptime(end, "%d-%m-%Y")
    return [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]    

In [30]:
def scrape_detik_finance(start, end=None, file_ke=1, path_file="data"):
    """Scraping untuk rentang tertentu. Hasilnya akan disimpan pada folder data
    masih dalam bentuk json karena akan dilakukan penyimpanan setiap 300 hari
    """
    
    generated_date = generate_date_range(start)
    hasil = []
    status = []
    status2 = None
    
    # Cek apakah sudah pernah melakukan scraping halaman detik finance sebelumnya
    # jika iya ambil file terakhir sehingga hasil scraping bisa berlanjut
    # dan tidak menimpa file yang sudah ada
    try:
        status2 = pd.read_csv(f'{path_file}/status_detik_finance.csv')
        file_ke = int(status2.file_ke.dropna().max()) + 1
    except:
        pass
    
    
    iterate = 1     
    for date in generated_date:
        tanggal_dmy = date.strftime("%d-%m-%Y")
        tanggal_mdy = date.strftime("%m/%d/%Y") # format url di detik finance m-d-y
        
        if len(hasil) == 0:
            tanggal_mulai = tanggal_dmy
        
        print(f"{iterate}. Mulai scraping tanggal {tanggal_dmy}")
        jumlah_berita = 0
        try:
            start_scraping = int(datetime.datetime.timestamp(datetime.datetime.now()))
            list_berita, jumlah_halaman = scrape_tanggal(tanggal_mdy)
            end_scraping = int(datetime.datetime.timestamp(datetime.datetime.now()))
            status_scrape = "berhasil"
            jumlah_berita = len(list_berita)

            hasil.append({ "tanggal" : tanggal_dmy,
                          "jumlah_berita": jumlah_berita,
                          "data" : list_berita})
            status.append({ "tanggal" : tanggal_dmy,
                           "status" : status_scrape,
                           "jumlah_berita": jumlah_berita,
                           "start_scrape" : start_scraping,
                           "end_scrape" : end_scraping,
                           "jumlah_halaman" : jumlah_halaman,
                           "file_ke" : file_ke})
        except:            
            status_scrape = "gagal"
            status.append({ "tanggal" : tanggal_dmy,
                           "status" : status_scrape,
                           "jumlah_berita": None,
                           "start_scrape" : None,
                           "end_scrape" : None,
                           "jumlah_halaman" : None,
                           "file_ke" : None})

        print(f"Scraping tanggal {tanggal_dmy} {status_scrape}, jumlah berita {jumlah_berita}\n")
        # setiap iterasi kelipatan 300, save datanya dan kosongkan variabel hasil
        # variabel hasil dikosongkan agar hemat memori dan ketika disimpan tidak 
        # membutuhkan waktu yang lama
        try:
            if iterate % 300 == 0:
                with open(f'{path_file}/json_detik_finance/{file_ke}_{tanggal_mulai}_{tanggal_dmy}.txt', 'w', encoding='utf-8') as f:
                    json.dump(hasil, f, ensure_ascii=False)
                hasil = []
                file_ke += 1
                s = pd.DataFrame(status)
                
                # gabungkan isi file status yang sudah ada dengan hasil scraping sekarang
                if status2 is None:
                    s = pd.concat([status2, s]).sort_values('tanggal')
                s.to_csv(f"{path_file}/status_detik_finance.csv")

            iterate += 1
        except:
            break

    # save iterasi yang terakhir
    try:
        with open(f'{path_file}/json_detik_finance/{file_ke}_{tanggal_mulai}_{tanggal_dmy}.txt', 'w', encoding='utf-8') as f:
            json.dump(hasil, f, ensure_ascii=False)

        s = pd.DataFrame(status)            
        if status2 is None:
            s = pd.concat([status2, s]).sort_values('tanggal')
        s.to_csv(f"{path_file}/status_detik_finance.csv")
    except:
        print("Terjadi error")
        
    return status, hasil

In [ ]:
status, hasil = scrape_detik_finance("01-04-2004")
status

## Satukan File
Nyatuin seluruh file yang ada di folder `data/json_detik finance` menjadi satu dataframe

In [41]:
status = pd.read_csv("data/status_detik_finance.csv", index_col=0)
status[status.status == 'gagal']

,tanggal,status,jumlah_berita,start_scrape,end_scrape,jumlah_halaman,file_ke
292,31-01-2014,gagal,NaN,NaN,NaN,NaN,NaN
332,12-03-2014,gagal,NaN,NaN,NaN,NaN,NaN
333,13-03-2014,gagal,NaN,NaN,NaN,NaN,NaN
340,20-03-2014,gagal,NaN,NaN,NaN,NaN,NaN
383,02-05-2014,gagal,NaN,NaN,NaN,NaN,NaN
394,13-05-2014,gagal,NaN,NaN,NaN,NaN,NaN
467,25-07-2014,gagal,NaN,NaN,NaN,NaN,NaN
677,20-02-2015,gagal,NaN,NaN,NaN,NaN,NaN
1095,13-04-2016,gagal,NaN,NaN,NaN,NaN,NaN


karena cuman sedikit kayaknya ga perlu deh di scriping ulang

In [9]:
files = listdir("data/json_detik_finance/")

data = None
for file_txt in files:
    try:
        with open(f"data/json_detik_finance/{file_txt}", "r", encoding="utf-8") as f:
            isi = json.load(f)
            f.close()
    except:
        with open(f"data/json_detik_finance/{file_txt}", "r") as f:
            isi = json.load(f)
            f.close()
    
    if data is None:
        data = pd.DataFrame(isi)
    else:
        data = pd.concat([data, pd.DataFrame(isi)])
        
data

,tanggal,jumlah_berita,data
0,23-08-2011,70,[{'judul': 'SBY Belum Tentukan Pengganti Semen...
1,24-08-2011,69,[{'judul': 'Pasokan Dana ATM BRI Aman Hingga L...
2,25-08-2011,58,[{'judul': 'BKPM Yakin Tax Holiday Sedot Banya...
3,26-08-2011,54,[{'judul': 'Pemerintah Siapkan 13.000 Km Jalur...
4,27-08-2011,14,"[{'judul': 'Menteri ESDM Tinjau PLTA Cirata', ..."
...,...,...,...
295,18-08-2011,57,[{'judul': 'Dubes Havas Dapat Kehormatan Buka ...
296,19-08-2011,68,[{'judul': 'Semen Gresik Bingung Pasokan di Su...
297,20-08-2011,18,[{'judul': 'Pertamina Siap 'Belanja' Rp 359 Tr...
298,21-08-2011,10,"[{'judul': 'Sukseskan MP3EI, Jasa Marga Garap ..."


In [10]:
# pisahkan kolom data menjadi beberapa kolom
data_full = None

for index, row in data.iterrows():
    data_satu_hari = pd.DataFrame(row['data'])
    data_satu_hari['tanggal'] = row['tanggal']
    
    if data_full is None:
        data_full = pd.DataFrame(data_satu_hari)
    else:
        data_full = pd.concat([data_full, pd.DataFrame(data_satu_hari)])

data_full = data_full.reset_index(drop=True)
data_full

,judul,sub_judul,kategori,timestamp_publish,link,tanggal
0,SBY Belum Tentukan Pengganti Sementara Menteri...,None,berita ekonomi bisnis,1314109401,https://finance.detik.com/berita-ekonomi-bisni...,23-08-2011
1,SBY Doakan Kesembuhan Untuk Mustafa,None,berita ekonomi bisnis,1314108854,https://finance.detik.com/berita-ekonomi-bisni...,23-08-2011
2,Agus Marto Tak Mau TPPI Dipailitkan,None,energi,1314107263,https://finance.detik.com/energi/d-1709669/agu...,23-08-2011
3,Kemenperin Mengaku Butuh 2.500 PNS Baru,None,berita ekonomi bisnis,1314106530,https://finance.detik.com/berita-ekonomi-bisni...,23-08-2011
4,"Adaro 'Caplok' Mustika Indah US$ 222,5 Juta",None,bursa dan valas,1314106004,https://finance.detik.com/bursa-dan-valas/d-17...,23-08-2011
...,...,...,...,...,...,...
340627,Pertamina Optimistis Proyek Senoro Rampung 2014,None,energi,1313973707,https://finance.detik.com/energi/d-1707913/per...,22-08-2011
340628,Garuda Terus Kejar Piutang Merpati Rp 330 Miliar,None,berita ekonomi bisnis,1313970218,https://finance.detik.com/berita-ekonomi-bisni...,22-08-2011
340629,Menteri ESDM Minta Mobil Pemudik Pakai Pertamax,None,energi,1313969568,https://finance.detik.com/energi/d-1707893/men...,22-08-2011
340630,"Gara-gara Demo, RI Kehilangan Minyak 2.000 Barel",None,energi,1313968165,https://finance.detik.com/energi/d-1707890/gar...,22-08-2011


In [11]:
# selected_kategori = ["bursa dan valas", "market research", "finansial", "fintech", "industri", "energi"]
# data_selected = data_full[data_full.kategori.isin(selected_kategori)].reset_index(drop=True)
data_selected = data_full
data_selected

,judul,sub_judul,kategori,timestamp_publish,link,tanggal
0,SBY Belum Tentukan Pengganti Sementara Menteri...,None,berita ekonomi bisnis,1314109401,https://finance.detik.com/berita-ekonomi-bisni...,23-08-2011
1,SBY Doakan Kesembuhan Untuk Mustafa,None,berita ekonomi bisnis,1314108854,https://finance.detik.com/berita-ekonomi-bisni...,23-08-2011
2,Agus Marto Tak Mau TPPI Dipailitkan,None,energi,1314107263,https://finance.detik.com/energi/d-1709669/agu...,23-08-2011
3,Kemenperin Mengaku Butuh 2.500 PNS Baru,None,berita ekonomi bisnis,1314106530,https://finance.detik.com/berita-ekonomi-bisni...,23-08-2011
4,"Adaro 'Caplok' Mustika Indah US$ 222,5 Juta",None,bursa dan valas,1314106004,https://finance.detik.com/bursa-dan-valas/d-17...,23-08-2011
...,...,...,...,...,...,...
340627,Pertamina Optimistis Proyek Senoro Rampung 2014,None,energi,1313973707,https://finance.detik.com/energi/d-1707913/per...,22-08-2011
340628,Garuda Terus Kejar Piutang Merpati Rp 330 Miliar,None,berita ekonomi bisnis,1313970218,https://finance.detik.com/berita-ekonomi-bisni...,22-08-2011
340629,Menteri ESDM Minta Mobil Pemudik Pakai Pertamax,None,energi,1313969568,https://finance.detik.com/energi/d-1707893/men...,22-08-2011
340630,"Gara-gara Demo, RI Kehilangan Minyak 2.000 Barel",None,energi,1313968165,https://finance.detik.com/energi/d-1707890/gar...,22-08-2011


In [12]:
data_selected = data_selected.sort_values("timestamp_publish").reset_index(drop=True)
data_selected.to_csv("data/list_berita_detik.csv", index=False)

## Scraping Satu Halaman

In [2]:
data_judul = pd.read_csv("data/list_berita_detik.csv")
data_judul['tanggal'] = pd.to_datetime(data_judul.tanggal, format="%d-%m-%Y")
data_judul['tanggal'] = data_judul['tanggal'].dt.strftime('%Y-%m-%d')
data_judul

,judul,sub_judul,kategori,timestamp_publish,link,tanggal
0,OPEC Putuskan Kurangi Produksi,hasil sidang opec di wina,berita ekonomi bisnis,1080770940,https://finance.detik.com/berita-ekonomi-bisni...,2004-04-01
1,"Setelah Window Dressing, IHSG Rawan Profit Taking",NaN,bursa dan valas,1080787800,https://finance.detik.com/bursa-dan-valas/d-12...,2004-04-01
2,Rupiah Dibuka Menguat di Rp 8570,NaN,moneter,1080788280,https://finance.detik.com/moneter/d-122087/rup...,2004-04-01
3,"Greenspan Dikabarkan Kena Serangan Jantung, Do...",NaN,moneter,1080791460,https://finance.detik.com/moneter/d-122086/gre...,2004-04-01
4,Ditjen Pajak Bongkar Jual Beli Faktur Pajak Fi...,NaN,moneter,1080799020,https://finance.detik.com/moneter/d-122084/dit...,2004-04-01
...,...,...,...,...,...,...
340627,"UMKM Resah Produk Impor, Ketua MPR Minta Ada P...",NaN,berita ekonomi bisnis,1638892559,https://finance.detik.com/berita-ekonomi-bisni...,2021-12-07
340628,Siap-siap Tarif PBB Bakal Naik,infografis,infografis,1638892835,https://finance.detik.com/infografis/d-5845257...,2021-12-07
340629,Korban Erupsi Semeru Diusulkan Dapat Keringana...,NaN,berita ekonomi bisnis,1638893242,https://finance.detik.com/berita-ekonomi-bisni...,2021-12-07
340630,"Segera Diresmikan Jokowi, Menhub Cek Bandara T...",NaN,infrastruktur,1638895340,https://finance.detik.com/infrastruktur/d-5845...,2021-12-07


untuk memantau hasil scraping maka akan dibuat file status_isi_detik yang berisi index dan status berhasil scraping atau tidak misalnya scaraping tiba tiba gagal maka cukup melihat data di file tersebut untuk mengetahui sudah sampai mana proses scrapingnya sehingga tidak mulai dari awal lagi

In [ ]:
iterasi = 0
status = []
last_index = -1

try:
    data_status = pd.read_csv("data/status_isi_detik.csv", index_col=0)
    status = data_status.to_dict('records')
    last_index = data_status.index[-1]
except:
    pass

for tanggal, df, in data_judul.iloc[last_index +  1:, :].groupby("tanggal"):
    data_per_tanggal = []
    print(f"Scraping tanggal {tanggal}")
    
    for index, row in df.iterrows():
        tmp = {'index' : index, 'judul' : row['judul'], 'link' : row['link'], 'tanggal' : tanggal}
        print(f"Scraping link {tmp['link']}?single=1")
        
        try:
            URL = f"{tmp['link']}?single=1"
            page = requests.get(URL, headers={'user-agent': 'My app'})
            page = BeautifulSoup(page.content, 'html.parser')
            
            body_content = page.select(".detail__body-text.itp_bodycontent")[0]

            for div in body_content.find_all("div"):
                div.decompose()
            for table in body_content.find_all("table"):
                table.decompose()
            for style in body_content.find_all("style"):
                style.decompose()
            for child in body_content.children:
                if isinstance(child,Comment):
                    child.extract()

            tmp['is_sukses'] = True
            tmp['isi'] = body_content.text
            status.append({'index': index, "is_sukses" : True})
            
        except:
            tmp['is_sukses'] = False
            tmp['isi'] = None
            status.append({'index': index, "is_sukses" : False})
            time.sleep(3)
        
        data_per_tanggal.append(tmp)
        iterasi += 1
        
        # setiap iterasi ke 200 istirahat 15 detik
        if iterasi % 200 == 0:
            time.sleep(15)
    
    pd.DataFrame(status).to_csv("data/status_isi_detik.csv")
    try:
        with open(f'data/isi_berita_detik/{tanggal}.txt', 'w', encoding='utf-8') as f:
            json.dump(data_per_tanggal, f, ensure_ascii=False)
        print("\n")
    except:
        print("Terjadi error")
        break

## Scraping Unsuccess

In [18]:
status_isi = pd.read_csv('data/status_isi_detik.csv', index_col=0)
unsuccess = status_isi[status_isi.is_sukses == False].index
unsuccess

Int64Index([ 44418,  44423,  44452,  44467,  44475,  44520,  44527,  44538,
             44551,  44642,
            ...
            340521, 340532, 340563, 340572, 340585, 340606, 340612, 340613,
            340618, 340621],
           dtype='int64', length=15810)

In [19]:
len(unsuccess)

15810

In [28]:
data_judul.iloc[unsuccess, :].kategori.value_counts()

foto bisnis              13972
detiktv                   1776
bursa dan valas             36
kompetisi foto               8
berita ekonomi bisnis        7
12                           2
moneter                      2
kompetisi film               1
klinik ukm                   1
01                           1
10                           1
infrastruktur                1
06                           1
solusiukm                    1
Name: kategori, dtype: int64

setelah di cek untuk kategori `foto bisnis` halamannya berbeda dengan yang lain, class `itp_bodycontent` tidak ada dihalaman sehingga codingan scriping perlu disesuaikan untuk kategori `foto bisnis`. Sedangkan untuk kategori `detiktv` itu isinya hanya video, jadi bisa diabaikan. Untuk kategori lainnya juga linknya tidak valid lagi

In [ ]:
hasil_scrape = []
status = []
iterasi = 400

# ambil hanya kategori foto bisnis
foto_bisnis = data_judul.iloc[unsuccess, :]
foto_bisnis = foto_bisnis[foto_bisnis.kategori == "foto bisnis"]

for index, row in foto_bisnis.iterrows():
    tmp = {'index' : index, 'judul' : row['judul'], 'link' : row['link'], 'tanggal' : row['tanggal']}
    print(f"{index}. Scraping link {tmp['link']}?single=1")

    try:
        URL = f"{tmp['link']}?single=1"
        page = requests.get(URL, headers={'user-agent': 'My app'})
        page = BeautifulSoup(page.content, 'html.parser')

        # disini yang berbeda kalau di kodingan sebelumnya page.select(".detail__body-text.itp_bodycontent")
        body_content = page.select(".detail__body-text")[0]

        for div in body_content.find_all("div"):
            div.decompose()
        for table in body_content.find_all("table"):
            table.decompose()
        for style in body_content.find_all("style"):
            style.decompose()
        for child in body_content.children:
            if isinstance(child,Comment):
                child.extract()

        tmp['is_sukses'] = True
        tmp['isi'] = body_content.text
        status_isi.iloc[index, 1] = True
        print('success')
    except:
        tmp['is_sukses'] = False
        tmp['isi'] = None
        status_isi.iloc[index, 1] = False
        print('gagal')
        time.sleep(1)

    hasil_scrape.append(tmp)
    iterasi += 1
    if iterasi % 200 == 0:
        time.sleep(10)
        try:
            #setiap 200 iterasi simpan datanya ke file dengan penamaan hasil scrape unsuccess
            with open(f'data/isi_berita_detik/hasil_scrape_unsuccess_{iterasi}.txt', 'w', encoding='utf-8') as f:
                json.dump(hasil_scrape, f, ensure_ascii=False)
                hasil_scrape = []
            print("\n")
            status_isi.to_csv("data/status_isi_detik.csv")
        except:
            print("Terjadi error")
            break